In [139]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [140]:
repo_list = pd.DataFrame()
repo_list_query = f"""
SELECT repo_id, repo_name, repo_path from repo;
    """
repo_list = pd.read_sql_query(repo_list_query, con=engine)
print(repo_list)

      repo_id                            repo_name  \
0       26042                               S3JDBC   
1       27470       docs-dynatrace-fullstack-addon   
2       26235                            concourse   
3       27566  docs-datadog-application-monitoring   
4       26719                          spring-bian   
...       ...                                  ...   
2706    26973                               fiscal   
2707    27274                                 a11y   
2708    26130                 philips-hue-resource   
2709    27399            pcf-bosh-terraforming-gcp   
2710    25541                 rabbitmq-trust-store   

                                  repo_path  
0      github.com/Pivotal-Data-Engineering/  
1                    github.com/pivotal-cf/  
2                  github.com/pcfdev-forks/  
3                    github.com/pivotal-cf/  
4     github.com/Pivotal-Field-Engineering/  
...                                     ...  
2706            github.com/pi

In [141]:
#from datetime import date
import datetime 

current = datetime.date.today()
today = "'" + str(current) + "'"
#print(today)

first_current = current.replace(day=1)
last_month = first_current - datetime.timedelta(days=1)
end_date = "'" + str(last_month) + "'"
#print(end_date)

start = last_month - datetime.timedelta(days=365) #### 6 MONTHS
start_date = "'" + str(start) + "'"

print(start_date, end_date)

'2019-03-01' '2020-02-29'


In [142]:
    repo_list_commits = pd.DataFrame()
    repo_list_commits_query = f"""
            SELECT repo.repo_id, repo.repo_name, repo.repo_path, repo.forked_from, COUNT(commits.cmt_id) from repo, commits
            WHERE 
                repo.repo_id = commits.repo_id
                AND commits.cmt_author_timestamp >= {start_date}
                AND commits.cmt_author_timestamp <= {end_date}
            GROUP BY repo.repo_id
            ORDER BY COUNT(commits.cmt_id);
            """
    repo_list_commits = pd.read_sql_query(repo_list_commits_query, con=engine)
    print(repo_list_commits.tail(5))

      repo_id                 repo_name                    repo_path  \
1134    27050                    harbor          github.com/pivotal/   
1135    27497  azure-blobstore-resource       github.com/pivotal-cf/   
1136    25760               spring-boot  github.com/Spring-projects/   
1137    25663          spring-framework  github.com/Spring-projects/   
1138    27645                   cvelist       github.com/pivotal-cf/   

     forked_from   count  
1134        None   17314  
1135        None   21553  
1136        None   27999  
1137        None   33921  
1138        None  215517  


In [194]:
    repo_list_commits = pd.DataFrame()
    repo_list_commits_query = f"""
            SELECT count(distinct commits.cmt_commit_hash), repo.repo_id, repo.repo_name, repo.repo_path from repo, commits
            WHERE 
                repo.repo_id = commits.repo_id
                AND commits.cmt_author_timestamp >= {start_date}
                AND commits.cmt_author_timestamp <= {end_date}
            GROUP BY repo.repo_id
            ORDER BY count(distinct commits.cmt_commit_hash);
            """
    repo_list_commits = pd.read_sql_query(repo_list_commits_query, con=engine)
    repo_list_commits
    #repo_list_commits.loc[repo_list_commits.repo_id == 28031]
    #df.loc[df.a == 3
    #.loc["California","2013"]

,count,repo_id,repo_name,repo_path
0,1,26977,pks-windows-aquamarker,github.com/pivotalservices/
1,1,26497,edu-store-redirect,github.com/platform-acceleration-lab/
2,1,26499,spring-cloud-developer-code-v2.0.5,github.com/platform-acceleration-lab/
3,1,25694,spring-flex-roo,github.com/Spring-projects/
4,1,28131,mockups,github.com/concourse/
...,...,...,...,...
1134,2454,25663,spring-framework,github.com/Spring-projects/
1135,2556,28051,concourse,github.com/concourse/
1136,4057,28118,charts,github.com/concourse/
1137,4892,25760,spring-boot,github.com/Spring-projects/


In [78]:
top = repo_list_commits.loc[repo_list_commits['count'] > 2400]

In [79]:
len(top.repo_id)

20

In [80]:
top

,repo_id,repo_name,repo_path,forked_from,count
791,25832,spring-data-geode,github.com/Spring-projects/,None,2462
792,28041,velero-plugin-for-gcp,github.com/vmware-tanzu/,None,2760
793,28024,terraform-provider-vmc,github.com/vmware/,None,2960
794,26000,steeltoe,github.com/SteeltoeOSS/,None,3192
795,28030,sonobuoy,github.com/vmware-tanzu/,None,3422
796,25857,gpdb,github.com/greenplum-db/,None,3945
797,27003,deck,github.com/pivotal/,None,4193
798,27913,clarity,github.com/vmware/,None,4264
799,28051,concourse,github.com/concourse/,None,4412
800,28046,velero-plugin-for-vsphere,github.com/vmware-tanzu/,None,4428


In [81]:
    for index, repo in top.iterrows():
        print(repo['repo_id'], repo['repo_name'])

25832 spring-data-geode
28041 velero-plugin-for-gcp
28024 terraform-provider-vmc
26000 steeltoe
28030 sonobuoy
25857 gpdb
27003 deck
27913 clarity
28051 concourse
28046 velero-plugin-for-vsphere
28031 velero
28037 octant
28118 charts
27050 harbor
25663 spring-framework
25760 spring-boot
27043 kpack
25726 spring-security
28033 watch-proxy
27645 cvelist


In [169]:
    commit_exp = pd.DataFrame()
    commit_exp_query = f"""
            SELECT DISTINCT cmt_commit_hash, commits.cmt_author_timestamp from commits
            WHERE 
                commits.repo_id = 26983
                AND commits.cmt_author_timestamp >= '2019-07-01'
                AND commits.cmt_author_timestamp <= '2019-08-01'
            ORDER BY commits.cmt_author_timestamp;
            """
    commit_exp = pd.read_sql_query(commit_exp_query, con=engine)
    print(commit_exp)

                             cmt_commit_hash      cmt_author_timestamp
0   7873a33622056c7f88d6c84dd505cd7667eca1de 2019-07-11 14:58:46+00:00
1   6f2121abd49dfdbe443450a6c707b8d45fcf8efc 2019-07-11 15:10:08+00:00
2   3308c09aba50edf328b84c9efd10077538bcead0 2019-07-11 15:20:07+00:00
3   9567149ccf4f007c207d55fe1790ae0d17007a39 2019-07-11 15:34:19+00:00
4   8250e2909e7f1ce19bc4ed8296a10a1174ab5ec3 2019-07-11 15:36:12+00:00
..                                       ...                       ...
92  0fc6028f9b965c3c857eb31a9a2ef56812dd9795 2019-07-27 15:15:11+00:00
93  6d99a22e25106e91bca4f96fd7bc9a79ba6c6ce3 2019-07-29 15:56:15+00:00
94  f5cce9ac4a968a0e440c36d410a621412dddf15a 2019-07-29 18:43:12+00:00
95  2e31d5d424f90517e01fe17e3895ac409a8d752b 2019-07-30 09:28:22+00:00
96  3c6d4b6a8b12b33614871c50108a11682d3177fe 2019-07-30 10:43:49+00:00

[97 rows x 2 columns]


In [161]:
    commit_exp = pd.DataFrame()
    commit_exp_query = f"""
            SELECT updated_at, repo_id from repo
            ORDER BY updated_at
            """
    commit_exp = pd.read_sql_query(commit_exp_query, con=engine)
    print(commit_exp)

     updated_at  repo_id
0          None    26042
1          None    27470
2          None    26235
3          None    27566
4          None    26719
...         ...      ...
2706       None    26973
2707       None    27274
2708       None    26130
2709       None    27399
2710       None    25541

[2711 rows x 2 columns]
